In [1]:
import sys

# Add Robyn to path
sys.path.append("/Users/yijuilee/robynpy_release_reviews/Robyn/python/src")

In [2]:
# Test Pareto Optimizer

from robyn.modeling.pareto.pareto_optimizer import ParetoOptimizer

from utils.data_mapper import import_output_models, import_input_collect, load_data_from_json

/Users/yijuilee/robynpy_release_reviews/robynvenv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# Load data from JSON
inputCollect = load_data_from_json(
    "/Users/yijuilee/project_robyn/original/Robyn_original_2/Robyn/robyn_api/data/Pareto_InputCollect.json"
)
outputModel = load_data_from_json(
    "/Users/yijuilee/project_robyn/original/Robyn_original_2/Robyn/robyn_api/data/Pareto_OutputModels.json"
)
input_collect = import_input_collect(inputCollect)
model_outputs = import_output_models(outputModel)
display((model_outputs.hyper_bound_ng))

Debug: R output data keys: ['trial1', 'trial2', 'trial3', 'trial4', 'trial5']
Data for trial1: Keys = ['resultCollect', 'hyperBoundNG', 'hyperBoundFixed']
Debug: resultCollect keys for trial1: ['resultHypParam', 'xDecompAgg', 'liftCalibration', 'decompSpendDist', 'iter', 'elapsed.min']
Debug: Sample resultHypParam for trial1: [{'facebook_S_alphas': 1.98202923325, 'facebook_S_gammas': 0.58424338159, 'facebook_S_thetas': 0.09072730547999999, 'newsletter_alphas': 1.7514167265, 'newsletter_gammas': 0.6585122811199999, 'newsletter_thetas': 0.22826231587, 'ooh_S_alphas': 1.9350874957500002, 'ooh_S_gammas': 0.80499137605, 'ooh_S_thetas': 0.26847073588000003, 'print_S_alphas': 1.3471347205000002, 'print_S_gammas': 0.6602001640299999, 'print_S_thetas': 0.34398200833000003, 'search_S_alphas': 1.835435262, 'search_S_gammas': 0.6915214566899999, 'search_S_thetas': 0.11253588111, 'tv_S_alphas': 1.71477961025, 'tv_S_gammas': 0.6107644945299999, 'tv_S_thetas': 0.6227415716, 'train_size': 0.6185610811

,facebook_S_alphas,facebook_S_gammas,facebook_S_thetas,newsletter_alphas,newsletter_gammas,newsletter_thetas,ooh_S_alphas,ooh_S_gammas,ooh_S_thetas,print_S_alphas,print_S_gammas,print_S_thetas,search_S_alphas,search_S_gammas,search_S_thetas,tv_S_alphas,tv_S_gammas,tv_S_thetas,train_size,lambda
0,0.5,0.3,0.0,0.5,0.3,0.1,0.5,0.3,0.1,0.5,0.3,0.1,0.5,0.3,0.0,0.5,0.3,0.3,0.5,0
1,3.0,1.0,0.3,3.0,1.0,0.4,3.0,1.0,0.4,3.0,1.0,0.4,3.0,1.0,0.3,3.0,1.0,0.8,0.8,1


In [4]:
mmm_data = input_collect["mmm_data"]
# display(mmm_data.data.head())
# Display Model Outputs

model_outputs = model_outputs
# display((model_outputs.trials[0].result_hyp_param))

hyperparameters = input_collect["hyperparameters"]
# display(hyperparameters)

featurized_mmm_data = input_collect["featurized_mmm_data"]

holidays_data = input_collect["holidays_data"]

In [5]:
# 3. Create ParetoOptimizer instance
pareto_optimizer = ParetoOptimizer(mmm_data, model_outputs, hyperparameters, featurized_mmm_data, holidays_data)

In [6]:
# 4. Run optimize function
pareto_result = pareto_optimizer.optimize(pareto_fronts="auto", min_candidates=100)

Pareto Optimization:   0%|                              | 0/100 [00:00<?, ?it/s]2024-11-14 02:18:50 [INFO] Starting model data aggregation
2024-11-14 02:18:50 [INFO] Computing Pareto fronts
2024-11-14 02:18:50 [INFO] Pareto front computation completed
Pareto Optimization:  50%|██████████          | 50/100 [00:00<00:00, 110.61it/s]2024-11-14 02:18:50 [INFO] Preparing Pareto data
2024-11-14 02:18:50 [INFO] Number of Pareto-optimal solutions found: 637
2024-11-14 02:18:50 [INFO] Selected 10 Pareto-fronts containing 102 candidates
2024-11-14 02:18:50 [INFO] Filtering data for selected Pareto fronts...
2024-11-14 02:18:50 [INFO] Calculating response curves for 510 models' media variables...
Pareto Optimization:  50%|██████████          | 50/100 [00:20<00:00, 110.61it/s]

In [ ]:
# 5. Check results
print("Pareto Optimization Results:")
print(f"Number of Pareto fronts: {pareto_result.pareto_solutions}")
print(f"MediaVecCollect: {pareto_result.media_vec_collect.shape, pareto_result.media_vec_collect}")
print("\Hyper parameter solutions:")
print(pareto_result.result_hyp_param)

print("\nAggregated decomposition results:")
print(pareto_result.x_decomp_agg)
print("\result Calibration:")
print(pareto_result.result_calibration)
print("\nx Decomp Vec Collect:")
print(pareto_result.x_decomp_vec_collect.shape, pareto_result.x_decomp_vec_collect)
print("\nCarryover percentage all:")
print(pareto_result.df_caov_pct_all.shape, pareto_result.df_caov_pct_all)
print("\Plot Data Collected")
print("NUMBER OF PLOTS Data collected for:", len(pareto_result.plot_data_collect["2_220_1"]))
print("Plot data for solid 3_206_6", pareto_result.plot_data_collect["2_220_1"])

# 6. Validate logic
assert pareto_result.pareto_fronts == "auto" or isinstance(
    pareto_result.pareto_fronts, int
), "Invalid pareto_fronts value"
assert not pareto_result.result_hyp_param.empty, "Empty result_hyp_param DataFrame"
assert not pareto_result.x_decomp_agg.empty, "Empty x_decomp_agg DataFrame"

print("\nAll assertions passed. The optimize function is working as expected.")

# Only export this to a pickle file if you plan to use the results and the mmm_data in another notebook

In [ ]:
import pickle

with open("pareto_result.pkl", "wb") as f:
    pickle.dump(pareto_result, f)
with open("mmmdata.pkl", "wb") as f:
    pickle.dump(mmm_data, f)